## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
%%time
import numpy as np
def massive_naive():
    B = []
    summa = 0
    for i in range(1000000):
        number = np.random.randint(0, 1000)
        B.append(number+100)
        summa += number+100  
    return summa/1000000
print(massive_naive())

599.80967
CPU times: user 4.65 s, sys: 154 ms, total: 4.81 s
Wall time: 4.91 s


In [2]:
%%time
def massive_numpy():
    A = np.random.uniform(0, 1000, 1000000)
    B = A + 100
    return np.mean(B)
print(massive_numpy())

600.0184357984164
CPU times: user 16.9 ms, sys: 8.57 ms, total: 25.4 ms
Wall time: 25.7 ms


In [20]:
from numba import jit, njit, vectorize, float64

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [34]:
!pip install line_profiler

     |████████████████████████████████| 96 kB 1.1 MB/s eta 0:00:01


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
import numpy as np
import pandas as pd
recipes = pd.read_csv("/Users/apple/Desktop/УНИК/TOBD/recipes_sample.csv", sep=',')
reviews = pd.read_csv("/Users/apple/Desktop/УНИК/TOBD/reviews_sample.csv", sep=',', index_col=0)

In [7]:
reviews

user_id  recipe_id        date  rating  \
370476        21752      57993  2003-05-01       5   
624300       431813     142201  2007-09-16       5   
187037       400708     252013  2008-01-10       4   
706134   2001852463     404716  2017-12-11       5   
312179        95810     129396  2008-03-14       5   
...             ...        ...         ...     ...   
1013457     1270706     335534  2009-05-17       4   
158736      2282344       8701  2012-06-03       0   
1059834      689540     222001  2008-04-08       5   
453285   2000242659     354979  2015-06-02       5   
691207       463435     415599  2010-09-30       5   

                                                    review  
370476   Last week whole sides of frozen salmon fillet ...  
624300   So simple and so tasty!  I used a yellow capsi...  
187037   Very nice breakfast HH, easy to make and yummy...  
706134   These are a favorite for the holidays and so e...  
312179   Excellent soup!  The tomato flavor is just gre...  
...                                                    ...  
1013457  This recipe was great! I made it last night. I...  
158736   This recipe is outstanding. I followed the rec...  
1059834  Well, we were not a crowd but it was a fabulou...  
453285   I have been a steak eater and dedicated BBQ gr...  
691207   Wonderful and simple to prepare seasoning blen...  

[126696 rows x 5 columns]

In [8]:
print("Привезенцев Семен")

Привезенцев Семен


In [9]:
recipes.submitted = recipes.submitted.astype("datetime64[ns]")
recipes.dtypes

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

In [10]:
print("Привезенцев Семен")

Привезенцев Семен


In [11]:
reviews.date = reviews.date.astype("datetime64[ns]")
reviews.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
rating                int64
review               object
dtype: object

In [12]:
print("Привезенцев Семен")

Привезенцев Семен


In [13]:
%%time
def df_iterrows_source_table():
    k = 0
    summa = 0
    for ind,row in reviews.iterrows():
        if row.date.year == 2010:
            k += 1
            summa += row.rating
    print("Среднее значение rating с использованием метода DataFrame.iterrows исходной таблицы:", summa/k)
df_iterrows_source_table()

Среднее значение rating с использованием метода DataFrame.iterrows исходной таблицы: 4.4544402182900615
CPU times: user 6.77 s, sys: 89.2 ms, total: 6.86 s
Wall time: 6.96 s


In [14]:
print("Привезенцев Семен")

Привезенцев Семен


In [15]:
%%time
def df_iterrows_2010_only():
    summa = 0
    reviews_2010 = reviews[reviews.date.dt.year == 2010]
    for ind,row in reviews_2010.iterrows():
        summa += row.rating
    print("Среднее значение rating с использованием метода DataFrame.iterrows только 2010 год:", summa/len(reviews_2010))
df_iterrows_2010_only()

Среднее значение rating с использованием метода DataFrame.iterrows только 2010 год: 4.4544402182900615
CPU times: user 645 ms, sys: 11.8 ms, total: 656 ms
Wall time: 666 ms


In [16]:
print("Привезенцев Семен")

Привезенцев Семен


In [17]:
%%time
def series_mean():
    reviews_2010 = reviews[reviews.date.dt.year == 2010]
    print("Среднее значение rating с помощью Series.mean:", reviews_2010.rating.mean())
series_mean()

Среднее значение rating с помощью Series.mean: 4.4544402182900615
CPU times: user 12.6 ms, sys: 1.98 ms, total: 14.6 ms
Wall time: 12.9 ms


In [18]:
print("Привезенцев Семен")

Привезенцев Семен


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [35]:
from line_profiler import LineProfiler

In [36]:
lp = LineProfiler()
lp_wrapper = lp(df_iterrows_source_table)
lp_wrapper()
lp.print_stats()

Среднее значение rating с использованием метода DataFrame.iterrows исходной таблицы: 4.4544402182900615
Timer unit: 1e-09 s

Total time: 12.8716 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1       1000.0   1000.0      0.0  
     3         1          0.0      0.0      0.0  
     4    126696 10015549000.0  79051.8     77.8  
     5    114602 2621131000.0  22871.6     20.4  
     6     12094    9608000.0    794.4      0.1  
     7     12094  225088000.0  18611.5      1.7  
     8         1     245000.0 245000.0      0.0  



In [37]:
print("Привезенцев Семен")

Привезенцев Семен


In [38]:
lp = LineProfiler()
lp_wrapper = lp(df_iterrows_2010_only)
lp_wrapper()
lp.print_stats()

Среднее значение rating с использованием метода DataFrame.iterrows только 2010 год: 4.4544402182900615
Timer unit: 1e-09 s

Total time: 1.24995 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1          0.0      0.0      0.0  
     3         1   17830000.0 17830000.0      1.4  
     4     12094  953891000.0  78873.1     76.3  
     5     12094  278054000.0  22991.1     22.2  
     6         1     172000.0 172000.0      0.0  



In [39]:
print("Привезенцев Семен")

Привезенцев Семен


In [40]:
lp = LineProfiler()
lp_wrapper = lp(series_mean)
lp_wrapper()
lp.print_stats()

Среднее значение rating с помощью Series.mean: 4.4544402182900615
Timer unit: 1e-09 s

Total time: 0.013665 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1   13078000.0 13078000.0     95.7  
     3         1     587000.0 587000.0      4.3  



In [41]:
print("Привезенцев Семен")

Привезенцев Семен


In [42]:
%%time
def df_iterrows_2010_only_faster():
    reviews_2010 = reviews[reviews.date.dt.year == 2010]
    print("Среднее значение rating с использованием метода DataFrame.iterrows только 2010 год:", reviews_2010.rating.sum()/len(reviews_2010))
df_iterrows_2010_only_faster()

Среднее значение rating с использованием метода DataFrame.iterrows только 2010 год: 4.4544402182900615
CPU times: user 14.8 ms, sys: 2.29 ms, total: 17.1 ms
Wall time: 13.1 ms


In [43]:
print("Привезенцев Семен")

Привезенцев Семен


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [44]:
%%time
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
get_word_reviews_count(reviews)



CPU times: user 22.8 s, sys: 431 ms, total: 23.2 s
Wall time: 23.9 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [45]:
%%time
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = 0
            word_reviews[word] += 1
    return word_reviews
get_word_reviews_count(reviews)


CPU times: user 10.4 s, sys: 144 ms, total: 10.6 s
Wall time: 11 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [46]:
print("Привезенцев Семен")

Привезенцев Семен


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [47]:
ratings = reviews.dropna(subset=['rating'])['rating']
ratings = ratings[ratings != 0].to_numpy()

In [48]:
%%time
def A(series):
    summ = 0
    n = 0
    for i in series:
        summ += i
        n += 1
    Ft = summ/n
    summ = 0
    for at in series:
        summ += abs((at-Ft)/at)
    return (summ*100)/n
A(ratings)

CPU times: user 440 ms, sys: 9.09 ms, total: 449 ms
Wall time: 459 ms


16.26666333876162

In [49]:
print("Привезенцев Семен")

Привезенцев Семен


In [50]:
@njit
def B(series):
    summ = 0
    n = 0
    for i in series:
        summ += i
        n += 1
    Ft = summ/n
    summ = 0
    for at in series:
        summ += abs((at-Ft)/at)
    return (summ*100)/n

In [51]:
%%time
B(ratings)

CPU times: user 383 ms, sys: 82.1 ms, total: 465 ms
Wall time: 1.7 s


16.26666333876162

In [52]:
print("Привезенцев Семен")

Привезенцев Семен


In [53]:
def C(series):
    Ft = np.mean(series)
    return ((abs(series - Ft)/series).sum()*100)/len(series)

In [54]:
%%time
C(ratings)

CPU times: user 2.3 ms, sys: 1.98 ms, total: 4.28 ms
Wall time: 3.94 ms


16.266663338799717

In [55]:
print("Привезенцев Семен")

Привезенцев Семен


In [56]:
Ft = np.mean(ratings)
@vectorize([float64(float64)])
def D(series):
    return (abs(series - Ft)/series)
def D_output(D):
    return (D.sum()*100)/len(D)

In [57]:
%%timeit
D_output(D(ratings))

241 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [58]:
print("Привезенцев Семен")

Привезенцев Семен
